In [ ]:
from common import *
from raster_file import RasterFile
use_hvplot()

In [ ]:
zone_data = cbrfc_zones()

In [ ]:
def plot_hv(first, second=None):
    diff = RasterFile(
        (ASO_DIR / f'Depth-Difference/AGU-2021/Depth_difference_{first}_50m.tif').as_posix()
    ).band_values()

    first =  hv.Violin(diff[zone_data == ALEC2HLF].compressed(), label='ALEC2HLF').opts(
        width=500, height=400,
        violin_fill_color='gold', title=f'ASO {first}'
    ) * \
    hv.Violin(diff[zone_data == ALEC2HMF].compressed(), label='ALEC2HMF'). \
        opts(violin_fill_color='teal') * \
    hv.Violin(diff[zone_data == ALEC2HUF].compressed(), label='ALEC2HUF'). \
        opts(violin_fill_color='indigo')
    
    if second is not None:
        diff = RasterFile(
            (ASO_DIR / f'Depth-Difference/AGU-2021/Depth_difference_{second}_50m.tif').as_posix()
        ).band_values()

        first = first + \
            hv.Violin(diff[zone_data == ALEC2HLF].compressed(), label='ALEC2HLF').opts(
                width=500, height=400,
                violin_fill_color='gold', title=f'ASO {second}'
            ) * \
            hv.Violin(diff[zone_data == ALEC2HMF].compressed(), label='ALEC2HMF'). \
                opts(violin_fill_color='teal') * \
            hv.Violin(diff[zone_data == ALEC2HUF].compressed(), label='ALEC2HUF'). \
                opts(violin_fill_color='indigo')
   
    return first

In [ ]:
plot_hv('20180331', '20180524')

In [ ]:
plot_hv('20190407', '20190610')

In [ ]:
plot_hv('20200214')

## Static plots 

In [ ]:
from matplotlib.ticker import AutoMinorLocator
from matplotlib.lines import Line2D
from datetime import datetime

In [ ]:
def plot_mpl(first, second):
    fig, axes = plt.subplots(ncols=2, sharey=True, dpi=300)
    ax_index = 0
    
    for day in [first, second]:
        diff = RasterFile(
            (ASO_DIR / f'Depth-Difference/AGU-2021/Depth_difference_{day}_50m.tif').as_posix()
        ).band_values()

        violin_position = [0.5, 1.0, 1.5]
        
        vp = axes[ax_index].violinplot([
                diff[zone_data == ALEC2HLF].compressed(),
                diff[zone_data == ALEC2HMF].compressed(),
                diff[zone_data == ALEC2HUF].compressed(),
            ],
            positions=violin_position,
            widths=0.4,
            showmeans=False,
            showmedians=True,
            showextrema=False,
            quantiles=[[0.05, 0.95], [0.05, 0.95], [0.05, 0.95]]
        )

        color = ['gold', 'teal', 'indigo']
        ci = 0
        for pc in vp['bodies']:
            pc.set_facecolor(color[ci])
            pc.set_edgecolor('Black')
            pc.set_lw(0.75)
            ci += 1

        for line in ['cquantiles', 'cmedians']:
            vp[line].set_color('black')
            vp[line].set_lw(0.75)

        vp['cmedians'].set_ls(':')
        vp['cmedians'].set_lw(1)

        axes[ax_index].set_xticks(violin_position, labels=['Lower', 'Middle', 'Upper'])
        
        date = datetime.strptime(day, '%Y%m%d')
        axes[ax_index].set_title(date.strftime('%d %B'), size=12, fontstyle='italic')
        
        ax_index += 1
    
    if first == '20180331':
        axes[0].set_yticks(np.arange(-2, 10, 2))
    elif first == '20190407':
        axes[0].set_yticks(np.arange(-4, 18, 2))
    axes[0].set_ylabel(r'$\Delta$ Snow Depth (m)')
    axes[0].yaxis.set_minor_locator(AutoMinorLocator())
    
    # fig.suptitle(r'$\Delta$ Snow Depth (ASO - iSnobal)')
    fig.suptitle(datetime.strptime(day, '%Y%m%d').strftime('%Y'), size=14)

In [ ]:
plot_mpl('20180331', '20180524')

In [ ]:
plot_mpl('20190407', '20190610')

In [ ]:
plt.figure(dpi=300)
plt.gca().legend(
    handles=[
        Line2D([0], [0], color='black', linestyle=':', label='Medians'),
        Line2D([0], [0], color='black', label='Quantiles (95%, 5%)')
    ],
    loc='center'
);

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable
from palettable.colorbrewer.diverging import RdBu_5 as RedBlueCmap

RED_BLUE_CMAP = RedBlueCmap.mpl_colormap

In [ ]:
def plot_area(day):
    differences = RasterFile(
        (ASO_DIR / f'Depth-Difference/AGU-2021/Depth_difference_{day}_50m.tif').as_posix()
    )
    
    data = differences.band_values()
    # data[data > OUTLIER] = np.ma.masked
    data[np.isnan(data)] = np.ma.masked

    fig, (ax1) = plt.subplots(1, 1, figsize=(8.7, 6.5), dpi=300)
    fig.set_facecolor('lightgrey')
    ax1.set_facecolor('slategrey')

    plt_data = ax1.imshow(
        data,
        vmin=-2,
        vmax=2,
        cmap=RED_BLUE_CMAP,
        extent=differences.extent
    )

    ax1.tick_params(
        axis='both', direction='inout', length=4, labelsize=8, pad=0
    )
    ax1.tick_params(axis='y', labelrotation=45)
    ax1.ticklabel_format(axis='both', style='plain')
    ax1.xaxis.set_ticks_position('both')
    ax1.yaxis.set_ticks_position('both')

    cax = make_axes_locatable(ax1).append_axes('right', size='5%', pad=0.15)
    cbar = fig.colorbar(
        plt_data,
        cax=cax,
        shrink=0.9,
        extend='both'
    )
    cbar.set_label(
        label=r'$\Delta$ Snow Depth (m)',
        labelpad=12,
        rotation=270,
    )
    ax1.set_title(datetime.strptime(day, '%Y%m%d').strftime('%d %B %Y'))


In [ ]:
plot_area('20180331')

In [ ]:
plot_area('20180524')

In [ ]:
plot_area('20190407')

In [ ]:
plot_area('20190610')